In [1]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csc
import os
import glob
import pickle
from tqdm import tqdm
import scipy

In [2]:
names = list(pd.read_csv("names.csv", header = None)[0])
Adj_mat = scipy.sparse.load_npz("adjacency_matrix.npz")
D_mat_half_inv = csc.csc_matrix(np.diag((1./np.sqrt(np.array(Adj_mat.sum(axis=0)))).flatten()))

D_V = csc.csc_matrix(np.eye(len(names)))
W_V = D_mat_half_inv@Adj_mat@D_mat_half_inv
L_V = D_V - W_V

In [3]:
output_save_dir = "output"
try:
    os.mkdir(output_save_dir)
except:
    print("Directory already created")

Directory already created


In [4]:
all_cancer_files = sorted(glob.glob("input/*.npz"))

cancer_names = [file.split("/")[1].split(".")[0].split("_")[0] for file in all_cancer_files][::2]

In [5]:
def nmf(X_mut, X_exp, K):
    norms = []
    delta_u_norms = []
    delta_v_norms = []
    residuals = []
    
    X_mut  = ((X_mut!=0)+0)
    
    X_exp = X_exp[:, np.ravel(np.sum(X_mut.todense(), axis=0))>0]
    Number_of_samples = X_mut.shape[0]

    corr = np.corrcoef(X_exp)
    sigma = 1.0
    W_U = np.exp(-((1.-corr)**2)/(2.*sigma**2))
    D_U = np.diag(np.sum(W_U,axis = 0))
    L_U = D_U - W_U

    lambda_LU = 1.0
    lambda_RU = 1.0
    lambda_LV = 1.0
    lambda_RV = 1.0
    eps_t = 10**(-5)
    
    U_init = np.maximum(np.random.rand(Number_of_samples,K),eps_t)
    U_prev = U_init@np.diag(np.sum(U_init,axis = 0)**(-1))
    V_init = np.maximum(np.random.rand(len(names),K),eps_t)
    V_prev = V_init@np.diag(np.sum(U_init, axis = 0))
    delta_U = 1.
    delta_V = 1.
    counter = 0
    
    while ((delta_U > 10**(-3)) or (delta_V > 10**(-3)) and (counter <= 500)):
        norms.append(np.linalg.norm(X_mut - U_prev@V_prev.T, ord="fro"))
        counter += 1
        V_part = X_mut.T@U_prev + lambda_LV*W_V@V_prev
        V_denum = V_prev@(U_prev.T@U_prev) + lambda_RV*np.ones((len(names),len(names)))@V_prev + lambda_LV*D_V@V_prev
        V_new = V_prev*V_part/(V_denum + eps_t)
        U_part = X_mut@V_new + lambda_LU*W_U@U_prev
        U_denum = U_prev@(V_new.T@V_new) + lambda_RU*U_prev + lambda_LU*D_U@U_prev
        U_new = U_prev*U_part/(U_denum + eps_t)
        norm_divide = np.sum(U_new,axis = 0)
        U_new = U_new@np.diag(norm_divide**(-1))
        V_new = V_new@np.diag(norm_divide)

        delta_U = np.linalg.norm(U_prev - U_new,'fro')**2/(np.linalg.norm(U_prev,'fro')**2)
        delta_V = np.linalg.norm(V_prev - V_new,'fro')**2/(np.linalg.norm(V_prev,'fro')**2)
        
        delta_u_norms.append(delta_U)
        delta_v_norms.append(delta_V)
        
        U_prev = U_new
        V_prev = V_new
        
    norms.append(np.linalg.norm(X_mut - U_prev@V_prev.T, ord="fro"))
    top_genes = list(pd.Series(names)[np.flip(np.argsort(np.sort(V_new, axis = 1)[:, -1]))[:10]])
    return top_genes, U_new, V_new, counter, [norms, delta_u_norms, delta_v_norms]

In [6]:
cancer2data = {}
for idx, file in enumerate(cancer_names):
    print("Running file "+file)
    X_mut = scipy.sparse.load_npz("input/"+file+"_mut.npz")
    X_exp = scipy.sparse.load_npz("input/"+file+"_exp.npz").todense()
    top_genes, U_new, V_new, counter, delta_arrs = nmf(X_mut, X_exp, 4)
    cancer2data[file] = [top_genes, U_new, V_new, counter, delta_arrs]
    res = {"top_genes": top_genes, "U_new":U_new, "V_new":V_new}
    with open(output_save_dir+"/"+file+".pk", 'wb') as f:
        pickle.dump(res, f)


Running file brca
Running file coadread
Running file gbm


In [7]:
top_genes_result = {name:cancer2data[name][0] for name in cancer_names}

In [8]:
top_genes_result

{'brca': ['PIK3CA',
  'TP53',
  'TTN',
  'MUC16',
  'PCDHA4',
  'PCDHA1',
  'PCDHA2',
  'PCDHA6',
  'PCDHA3',
  'CDH1'],
 'coadread': ['APC',
  'TP53',
  'TTN',
  'KRAS',
  'SYNE1',
  'MUC16',
  'LRP2',
  'PIK3CA',
  'LRP1B',
  'DMD'],
 'gbm': ['TP53',
  'PTEN',
  'EGFR',
  'NF1',
  'RB1',
  'PIK3R1',
  'ERBB2',
  'DST',
  'PIK3CA',
  'MSH6']}

In [9]:
from sklearn.decomposition import NMF

In [10]:
sklearn_model_regularized = {}
for idx, file in enumerate(cancer_names):
    X_mut = scipy.sparse.load_npz("input/"+file+"_mut.npz")
    X_mut  = ((X_mut!=0)+0)
    model = NMF(n_components=4, alpha=1.5,l1_ratio=0.66666666, random_state= 42)
    W = model.fit_transform(X_mut)
    H = model.components_
    top_genes = list(pd.Series(names)[np.flip(np.argsort(np.sort(H.T, axis = 1)[:, -1]))[:10]])
    sklearn_model_regularized[cancer_names[idx]] = top_genes

In [11]:
sklearn_model_regularized

{'brca': ['PCDHA1',
  'PCDHA2',
  'PCDHA3',
  'PCDHA4',
  'PCDHA5',
  'TTN',
  'PCDHA6',
  'PCDHA7',
  'PCDHA8',
  'PCDHA9'],
 'coadread': ['TTN',
  'APC',
  'SYNE1',
  'FAT4',
  'LRP2',
  'LRP1B',
  'DMD',
  'USH2A',
  'RELN',
  'NEB'],
 'gbm': ['PTEN',
  'TP53',
  'RB1',
  'NF1',
  'EGFR',
  'DOCK1',
  'ASPM',
  'KLF6',
  'ATR',
  'TNFRSF11B']}

In [12]:
sklearn_model_non_regularized = {}
for idx, file in enumerate(cancer_names):
    X_mut = scipy.sparse.load_npz("input/"+file+"_mut.npz")
    X_mut  = ((X_mut!=0)+0)
    model = NMF(n_components=4, alpha=0., random_state= 42)
    W = model.fit_transform(X_mut)
    H = model.components_
    top_genes = list(pd.Series(names)[np.flip(np.argsort(np.sort(H.T, axis = 1)[:, -1]))[:10]])
    sklearn_model_non_regularized[cancer_names[idx]] = top_genes

In [13]:
sklearn_model_non_regularized

{'brca': ['PCDHA1',
  'PCDHA2',
  'PCDHA3',
  'PCDHA4',
  'TTN',
  'PCDHA5',
  'PCDHA6',
  'PCDHA7',
  'PIK3CA',
  'PCDHA8'],
 'coadread': ['APC',
  'TTN',
  'SYNE1',
  'LRP1B',
  'DMD',
  'FAT4',
  'LRP2',
  'ABCA12',
  'RELN',
  'DNAH5'],
 'gbm': ['PTEN',
  'TP53',
  'RB1',
  'EGFR',
  'NF1',
  'DST',
  'PIK3R1',
  'COL1A1',
  'COL3A1',
  'MSH6']}

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(8,8))
plt.xlabel("Steps")
plt.ylabel("Residuals")
plt.title("Residuals for different types of cancer")
for file in cancer_names:
    plt.semilogy(cancer2data[file][4][0],'*-', label = file)
plt.legend();

In [15]:
plt.figure(figsize=(8,8))
plt.xlabel("Steps")
plt.ylabel("Delta U")
plt.title("Delta for U matrix")
for file in cancer_names:
    plt.semilogy(cancer2data[file][4][1],'*-', label = file)
plt.legend();

In [16]:
plt.figure(figsize=(8,8))
plt.xlabel("Steps")
plt.ylabel("Delta V")
plt.title("Delta for V matrix")
for file in cancer_names:
    plt.semilogy(cancer2data[file][4][2],'*-', label = file)
plt.legend();

In [17]:
brca_mut = scipy.sparse.load_npz("input/brca_mut.npz")
brca_exp = scipy.sparse.load_npz("input/brca_mut.npz").todense()

counts = []
svd_norms = []
nmf_norms = []
u, s, v = np.linalg.svd(brca_mut.todense(), full_matrices=True)
sigma = np.zeros((u.shape[0], v.shape[0]))
np.fill_diagonal(sigma, s)

for i in tqdm(range(1,970, 20)):
    _, u_brca, v_brca, cnt, _ = nmf(brca_mut, brca_exp, i)
    nmf_norms.append(np.linalg.norm(brca_mut.todense() - u_brca@(v_brca.T)))
    svd_norms.append(np.linalg.norm(brca_mut.todense() - u[:, :i]@sigma[:i, :i]@v[:i, :]))
    counts.append(cnt)

100%|██████████| 49/49 [14:44<00:00, 18.04s/it]


In [18]:
plt.figure(figsize=(8,8))
plt.xlabel("Number of vectors")
plt.ylabel("Number of steps")
plt.title("Dependency number of steps for different K")
plt.plot(list(range(1,970,20)), counts, '-*');